In [ ]:
from pathlib import Path
import pickle
from games.smb1Py.py_mario_bros.PythonSuperMario_master.source import tools
import games.smb1Py.py_mario_bros.PythonSuperMario_master.source.constants as c
from games.smb1Py.py_mario_bros.PythonSuperMario_master.source.states.segment import Segment, SegmentState
from play_level import LevelCheckpoint, annulus;
import numpy as np
from skimage.io import imshow

In [ ]:
checkpoint_loc = "level_routing_checkpoints/level1.chp";
level_path = Path('levels')/'testing'/'test1.lvl';

level = SegmentState(None,None,file_path=level_path);

with open(checkpoint_loc,'rb') as f:
    check:LevelCheckpoint = pickle.load(f);

preds = check.pred;
costs = check.costs;
rhs = check.d['rhs'];
g = check.d['g'];

game = tools.Control()
state_dict = {c.LEVEL: Segment()}
game.setup_states(state_dict, c.LEVEL)
game.state.startup(0,{c.LEVEL_NUM:1},initial_state=level);

search_data_resolution=4
task_offset_downscale=1
view_distance = 3.75;
tile_scale = 2;

gdat = game.get_game_data(view_distance,tile_scale);
mdat = game.get_map_data(search_data_resolution);

player_start:tuple[float,float] = gdat['pos'];
search_grids = [np.array(mdat[g]) for g in ['collision_grid']];
grid_size = search_grids[0].shape;
grids_bounds:tuple[int,int,int,int] =  mdat['grid_bounds'];

def get_cost(start:tuple[int,int],task:tuple[int,int]):
    # if mdat['collision_grid'][task[0]][task[1]] != 0:
    #     return float('inf');
    if (not in_bounds(task)):
        return float('inf');
    if (start,task) in costs:
        return costs[start,task];
    elif (start,task) not in preds:
        if mdat['collision_grid'][task[0]][task[1]] != 0:
            return float('inf');
        else:
            return(get_cost(start,(task[0]-1,task[1]-1)));
        raise Exception("Error: cannot evaluate fixed net outside of play conditions");
        self.predictions[start,task] = cost_from_fitness(dist(*grid_index_to_pos(start),grid_index_to_pos(task))*self.eval_fixed_net(search_grids,start,task,grid_size));
    return preds[start,task];

min_dist = 8; max_dist = 40
task_offsets = [(s[0]*task_offset_downscale,s[1]*task_offset_downscale) for s in annulus(min_dist/task_offset_downscale,max_dist/task_offset_downscale)]

def get_costs(pos)->np.ndarray:
    out_grid = np.ndarray(grid_size);
    out_grid.fill(0);
    # center = (max_dist,max_dist);
    for offset in task_offsets:
        task = (pos[0]+offset[0],pos[1]+offset[1]);
        if (in_bounds(task) and (pos,task) in costs):
            out_grid[task] = costs[pos,task]+0.2;
    out_grid[out_grid==float('inf')] = 0.1;
    return out_grid;

def get_preds(pos)->np.ndarray:
    out_grid = np.ndarray(grid_size);
    out_grid.fill(0);
    # center = (max_dist,max_dist);
    for offset in task_offsets:
        task = (pos[0]+offset[0],pos[1]+offset[1]);
        if (in_bounds(task) and (pos,task) in preds):
            out_grid[task] = preds[pos,task]+0.2;
    out_grid[out_grid==float('inf')] = 0.1;
    return out_grid;

def get_heuristic(pos)->np.ndarray:
    out_grid = np.ndarray(grid_size);
    out_grid.fill(0);
    # center = (max_dist,max_dist);
    for offset in task_offsets:
        task = (pos[0]+offset[0],pos[1]+offset[1]);
        if (in_bounds(task) and (pos,task) in rhs):
            out_grid[task] = rhs[pos,task]+0.2;
    out_grid[out_grid==float('inf')] = 0.1;
    return out_grid;

def in_bounds(pos:tuple[int,int]):
    return pos[0] >= 0 and pos[0] < grid_size[0] and pos[1] >= 0 and pos[1] < grid_size[1];

def pos_to_grid_index(pos:tuple[float,float]):
    pos = pos[0]-c.TILE_SIZE/(2*search_data_resolution),pos[1]-c.TILE_SIZE/(2*search_data_resolution);
    # print(f'pos:{pos}');
    position_parameter = ((pos[0]-grids_bounds[0])/(grids_bounds[1]-grids_bounds[0]),(pos[1]-grids_bounds[2])/(grids_bounds[3]-grids_bounds[2]));
    # print(position_parameter);
    closest_pixel = (round(position_parameter[0]*grid_size[0]),round(position_parameter[1]*grid_size[1]))
    # print(closest_pixel);
    return closest_pixel;

def grid_index_to_pos(index:tuple[int,int]):
    return (c.TILE_SIZE/(2*search_data_resolution)+index[0]/grid_size[0]*(grids_bounds[1]-grids_bounds[0])+grids_bounds[0],c.TILE_SIZE/(2*search_data_resolution)+index[1]/grid_size[1]*(grids_bounds[3]-grids_bounds[2])+grids_bounds[2]);

In [ ]:
import random
for i in range(10000):
    p1 = (random.random()*100,random.random()*100)
    p2 = pos_to_grid_index(p1);
    p3 = grid_index_to_pos(p2);
    p4 = pos_to_grid_index(p3);
    if (p2!=p4):
        print(p1,2,3,4);

In [ ]:
print(([(key,val) for key,val in costs.items() if key[1][1]==46]));
key = ((2,26),(24,46));
# costs[key] = 300;
# print(g)
print(rhs[key[1]]);
print(preds[key]);
print(costs[key]);
print(costs[key]+rhs[key[1]]);
print(sorted([(t[0],t[1]+rhs[t[0][1]],t[1]) for t in costs.items() if t[0][0] == key[0]],key=lambda v: v[1]));

In [ ]:
def show_image(im):
    imshow(im.transpose())

In [ ]:
import matplotlib.pyplot as plt
# p2 = (415,255);
start = pos_to_grid_index(player_start);
print(player_start);
print(start);
a = get_costs(start);
b = get_preds(start);
d = get_heuristic(start);
print(a.shape);

cost_pad = max_dist+2;
bounds = [max(start[0]-cost_pad,0),min(start[0]+cost_pad,grid_size[0]-1),max(start[1]-cost_pad,0),min(start[1]+cost_pad,grid_size[1]-1)];
a = a[bounds[0]:bounds[1]+1,bounds[2]:bounds[3]+1];
b = b[bounds[0]:bounds[1]+1,bounds[2]:bounds[3]+1];
display_grid = np.array(search_grids[0])[bounds[0]:bounds[1]+1,bounds[2]:bounds[3]+1];
display_grids:np.ndarray = np.concatenate(([display_grid,a,b],),axis=0);

show_image(display_grids);
# print(np.min(a),np.max(a));

In [ ]:
print(np.min(a[a>0.1]));
show_image(np.array((a,a,a))/np.max(a))

In [ ]:
print(np.min(b[b>0.21]))
print(np.count_nonzero(b>0.21))
show_image(np.array((b,b,b))/np.max(b))

In [ ]:
### Initialize D* Searcher (run in reverse through the level)
from typing import Literal

from tqdm import tqdm

from search import DStarSearcher, LevelSearcher

goal = [(48*c.TILE_SIZE,i*c.TILE_SIZE) for i in range(20)]; 

def d_star_heuristic(p1:tuple[int,int]|Literal['goal'],p2:tuple[int,int]|Literal['goal']): #made to be bidirectional since lazy
    if isinstance(p1,str):
        try:
            return min([d_star_heuristic(pos,p2) for pos,_ in d_star_pred(p1)]);
        except Exception as e:
            print(p1)
            print(list(d_star_succ(p1)));
            raise e;
    if isinstance(p2,str):
        return d_star_heuristic(p2,p1);

    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5;

def d_star_succ(pos:tuple[int,int]|Literal['goal']): #returns list of (succ,cost(pos,succ))
    if (isinstance(pos,str)):
        raise Exception("cannot find successors of the goal");
    # log("d_preds");
    for offset in tqdm(task_offsets):
        task = (pos[0]+offset[0],pos[1]+offset[1]);
        if (in_bounds(task)):
            yield (task,get_cost(pos,task));
    
    if pos in goal_idxs:
        yield ('goal',0);

def d_star_pred(pos:tuple[int,int]|Literal['goal']): #returns list of (pred,cost(pred,pos))
    # log("d_succs");
    if pos == 'goal':
        for i in goal_idxs:
            yield i,0;
        return;

    for offset in tqdm(task_offsets):
        task = (pos[0]+offset[0],pos[1]+offset[1]);
        if (in_bounds(task)):
            yield (task,get_cost(task,pos));

start_idx = pos_to_grid_index(player_start);
goal_idxs = [pos_to_grid_index(g) for g in goal];


d_searcher = DStarSearcher[tuple[int,int]|Literal['goal']](d_star_heuristic,start=start_idx,goal='goal',pred=d_star_pred,succ=d_star_succ,checkpoint=check.d);

### Initialize A* Searcher (runs the level forward, selects training data)

def a_star_heuristic(path:tuple[tuple[int,int],...]):
    d_searcher.explore_from_position(path[-1]);
    return d_searcher.rhs[path[-1]];

def a_star_succ(path:tuple[tuple[int,int],]):
    end = path[-1];
    for offset in task_offsets:
        new = (end[0]+offset[0],end[1]+offset[1]);
        if not in_bounds(new):
            continue;
        newpath = list(path);
        newpath.append(new);
        yield tuple(newpath);

def a_star_cost(p1:tuple[tuple[int,int],...],p2:tuple[tuple[int,int],...]):
    return get_cost(p1[-1],p2[-1]);

a_searcher = LevelSearcher[tuple[tuple[int,int],...],tuple[int,int]]((start_idx,),lambda x: x[-1] in goal_idxs,a_star_heuristic,lambda node: node[-1],a_star_succ,a_star_cost,frustration_multiplier=0.02,checkpoint=check.a);

In [ ]:
print(search_data_resolution)
print(c.TILE_SIZE)
t1 = (grid_index_to_pos((2,2)))
t2 = pos_to_grid_index(t1);
print(t1);
print(t2)
print(pos_to_grid_index((26.0,26.0)))

In [ ]:
f = max(a_searcher.frustration.values());
print(f)

In [ ]:
all_edges = [(t,a_searcher.sort_key(t),a_searcher.frustration[t[1]]) for t in a_searcher.open_edges];
all_edges.sort(key=lambda k:k[1]);
print("\n".join([str(t) for t in all_edges]));

In [ ]:
best_edges = a_searcher.sorted_edges()[:30];

In [ ]:
print("\n".join([str(s) for s in best_edges]));
print(a_searcher.sort_key(best_edges[0]));
print(a_searcher.sort_key((((1,25),), ((1,25),(5,1)))))